In [39]:
import sys
sys.path.append('./models/')

import torch
import torch.nn.functional as F
import numpy as np
import sounddevice as sd # 마이크 입력을 위한 라이브러리

In [40]:
MODEL_PATH = "./result/best.pt"
CLASS_NAME = ["danger", "fire", "gas", "non", "tsunami"]
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

SAMPLE_RATE = 16000
DURATION = 1
NUM_SAMPLES = SAMPLE_RATE * DURATION

In [41]:
def preprocess(audio_np):
    # numpy 배열을 torch 텐서로 변환, 배치 차원 추가: (1, num_samples)
    waveform = torch.tensor(audio_np, dtype=torch.float32).unsqueeze(0)

    # 절대값 정규화
    waveform = waveform / (waveform.abs().max() + 1e-9)

    # 길이가 부족할 경우 뒤에 0을 채움
    if waveform.shape[1] < NUM_SAMPLES:
        waveform = F.pad(waveform, (0, NUM_SAMPLES - waveform.shape[1]))
    # 길이가 길 경우 자름
    else:
        waveform = waveform[:, :NUM_SAMPLES]

    return waveform # (1, NUM_SAMPLES)

def predict(model, audio_tensor, device=DEVICE, class_name=CLASS_NAME):
    audio_tensor = audio_tensor.to(device)

    with torch.no_grad():
        output_dict = model(audio_tensor)
        prob = output_dict["clipwise_output"] # 클래스별 확률 출력
        prob_np = prob.cpu().numpy()[0] # numpy 배열로 변환 (shape: [클래스 수])

        # 0.5를 기준으로 예측된 클래스 선택
        pred_tensor = (prob_np > 0.5).astype(int)
        pred_labels = [class_name[i] for i, p in enumerate(pred_tensor) if p == 1]

        # 어떤 클래스도 0.5 이상이 아니면 가장 확률 높은 클래스 하나 선택
        if not pred_labels:
            highest_prob_idx = int(np.argmax(prob_np))
            return [class_name[highest_prob_idx]], prob_np
        
        return pred_labels, prob_np # 예측된 라벨 리스트, 클래스별 확률 반환

In [42]:
model = torch.load(MODEL_PATH, weights_only=False)
model.to(DEVICE)
model.eval()
print("")

In [43]:
print("실시간 마이크 입력을 시작합니다.")

try:
    # 실시간 마이크 입력 루프
    while True:
        # 오디오를 실시간으로 마이크에서 녹음
        audio_np = sd.rec(frames=NUM_SAMPLES, # 녹음할 샘플 수 (1초 분량)
                          samplerate=SAMPLE_RATE, # 샘플링 레이트
                          channels=1, # 단일 채널 (모노)
                          dtype="float32" # float32 타입으로 녹음
                          )
        sd.wait() # 녹음 완료까지 대기

        audio_np = np.squeeze(audio_np, axis=-1) # 녹음된 오디오: (16000, 1) → (16000,)으로 reshape
        audio_tensor = preprocess(audio_np) # 전처리 → (1, 16000) 텐서로 변환
        pred_labels, class_prob = predict(model, audio_tensor) # 예측

        print(f"예측 결과: {', '.join(pred_labels)}")
        for i, name in enumerate(CLASS_NAME):
            print(f" - {name}: {class_prob[i]:.4f}")

except KeyboardInterrupt as e:
    print("\n실시간 마이크 입력을 종료합니다.\n", e)

실시간 마이크 입력을 시작합니다.
예측 결과: non
 - danger: 0.0017
 - fire: 0.0082
 - gas: 0.0488
 - non: 0.7734
 - tsunami: 0.0022
예측 결과: non
 - danger: 0.0000
 - fire: 0.0006
 - gas: 0.0002
 - non: 0.9957
 - tsunami: 0.0000
예측 결과: non
 - danger: 0.0014
 - fire: 0.0122
 - gas: 0.0372
 - non: 0.8853
 - tsunami: 0.0027
예측 결과: non
 - danger: 0.0073
 - fire: 0.0166
 - gas: 0.0385
 - non: 0.3666
 - tsunami: 0.0100
예측 결과: non
 - danger: 0.0048
 - fire: 0.0069
 - gas: 0.1005
 - non: 0.3697
 - tsunami: 0.0202
예측 결과: non
 - danger: 0.0184
 - fire: 0.0117
 - gas: 0.0311
 - non: 0.1713
 - tsunami: 0.0105
예측 결과: non
 - danger: 0.0025
 - fire: 0.0042
 - gas: 0.0313
 - non: 0.3166
 - tsunami: 0.0098
예측 결과: non
 - danger: 0.0021
 - fire: 0.0116
 - gas: 0.0769
 - non: 0.3237
 - tsunami: 0.0301
예측 결과: non
 - danger: 0.0031
 - fire: 0.0069
 - gas: 0.0657
 - non: 0.6187
 - tsunami: 0.0077
예측 결과: non
 - danger: 0.0020
 - fire: 0.0078
 - gas: 0.0376
 - non: 0.5383
 - tsunami: 0.0074
예측 결과: non
 - danger: 0.0036
 - fire: 0.0